In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

##  Names to scrape for details

In [ ]:
name_list = pd.read_json('data/names_list.json')

In [ ]:
name_list[name_list.name == 'Anne']

In [ ]:
# scrapeable_females = name_list.ix[(name_list.total_female > 4), 'name']
# scrapeable_males = name_list.ix[(name_list.total_male > 4), 'name']
# scrapeable_females.to_csv('data/scrapeable_females.csv', index=False, encoding='utf-8')
# scrapeable_males.to_csv('data/scrapeable_males.csv', index=False, encoding='utf-8')

## 1. Birth rates

In [ ]:
MIN_YEAR = 1880
MAX_YEAR = 2014

Load the file with detailed stats per name.

In [ ]:
details = pd.read_json('data/details.json')
names_with_details = details[details.has_details]

In [ ]:
names_with_details[names_with_details.name == 'Anne']

We're mostly interested in: 
<li> <code>name</code> </li>
<li> <code>gender</code> </li>
<li> <code>name_type</code>: whether it's a first name or a follow name </li>
<li> <code>value</code>: list with # births per year </li>
<li> <code>approximation</code>: list with approximated # births per year </li>
<li> <code>year</code>: array indexing value and approximation </li>

<p>
Unfortunately, value, approximation and year are lists contained in cells in the DataFrame, so we have to do some pandas magic (hidden in the function <code>get_births</code>) to unpack the data.

In [ ]:
def get_births(df, id_vars, cols, var_name='ii'):
    """
    Parse scraped data and get birth stats.
    
    :param df: DataFrame with scraped data
    :param id_vars: columns to use as identifiers
    :param cols: columns with values to unpack
    :param var_name: dummy variable name
    :returns DataFrame with unpacked data
    """
    births = pd.concat([convert_and_index(df, id_vars, var_name, p) for p 
                        in cols], axis=1)
    births.reset_index(inplace=True)
    births.drop(var_name, axis=1, inplace=True)
    return births


def convert_and_index(df, id_vars, var_name, value_name):
    """
    Wrapper function for convert_nested_to_long: also sets id_vars
    and var_name as index.
    
    :param df: DataFrame
    :param id_vars: List to use as identified variables
    :param var_name: Name to use for the variable column
    :param value_name: Column name to unpack.
    :returns DataFrame with column data in long format
    """
    converted = convert_nested_to_long(df, id_vars, var_name, value_name)
    return converted.set_index(id_vars + [var_name])


def convert_nested_to_long(df, id_vars, var_name, value_name):
    """
    Convert lists nested in cells in a column to a DataFrame in long
    format.
    
    :param df: DataFrame
    :param id_vars: List to use as identified variables
    :param var_name: Name to use for the variable column
    :param value_name: Column name to unpack.
    :returns DataFrame with column data in long format
    """
    indexed = df.set_index(id_vars)
    unnested = indexed[value_name].apply(pd.Series).reset_index()
    long_format = pd.melt(unnested, id_vars=id_vars, 
                          var_name=var_name, value_name=value_name)
    return long_format

In [ ]:
births = get_births(names_with_details, ['name', 'gender', 'name_type'],
                    ['year', 'value', 'approximation'])

The function <code>convert_nested_to_long</code> converts the columns <code>year</code>, <code>value</code> and <code>approximation</code> individually. The lists in the cells of the column are converted to a <code>Series</code>, effectively turning the 1D column with lists into a 2D <code>DataFrame</code>. The data is then transformed from wide to long format by melting the <code>DataFrame</code>. Setting the index with the name, gender and name type preserves these index values during the transformation

In [ ]:
temp_name = names_with_details[names_with_details.name == 'Aad']
temp_name.iloc[0]

In [ ]:
wide_name = temp_name.set_index(['name', 'gender', 'name_type'])['approximation'].apply(pd.Series)
wide_name

In [ ]:
pd.melt(wide_name.reset_index(), id_vars=['name', 'gender', 'name_type'], 
        var_name='ii', value_name='approximation').head()

The processsed result looks like:

In [ ]:
births.head()

Approximate births are given for years that don't have exact data, so we would like to use approximations if no real values are given. 

In [ ]:
# Combine approximate & exact stats 
births['n_born'] = births.approximation
has_value = births.value > 0 
births.ix[has_value, 'n_born'] = births.ix[has_value, 'value']
# Tidy up
births.drop(['value', 'approximation'], axis=1, inplace=True)
births['year'] = births['year'].astype(int)
births.rename(columns={'year': 'birth_year'}, inplace=True)

To compute the sums of the births for both name types or both genders, group over the all other identifying columns, and sum the number of births. Append the resulting <code>DataFrame</code> to the original one.

In [ ]:
def append_sums(df, id_vars, value_vars, id_name, id_value):
    """
    Append sum aggregates.
    
    :param df: DataFrame
    :param id_vars: Column(s) to use as identifier variables
    :param value_vars: Column(s) to sum
    :param agg_name: Column to aggregate 
    :param agg_value: Value to use as identifier in aggregated column
    :returns DataFrame with aggregated sums appended
    """
    totals = df.groupby(id_vars, as_index=False)[value_vars].sum()
    totals[id_name] = id_value
    df = df[df[id_name] != id_value].append(totals, ignore_index=True)
    return df

In [ ]:
births = append_sums(births, ['name', 'gender', 'birth_year'],
                     'n_born', 'name_type', 'total')
births = append_sums(births, ['name', 'name_type', 'birth_year'],
                     'n_born', 'gender', 'any')

The end result looks like:

In [ ]:
births.head()

And we can already plot the yearly birth rates for a name:

In [ ]:
NAME = 'Anne'
id_cols = ['name_type', 'gender']
current_births = births[births.name == NAME]

fig, ax = plt.subplots(3, 3, figsize=(10, 10), sharex=True, sharey=True)
ax = ax.reshape(-1)  # Reshape the list for easy iteraton
for i_plot, (key, group) in enumerate(current_births.groupby(id_cols)):
    group.plot(x='birth_year', y='n_born', ax=ax[i_plot],
               title=' '.join(key), rot=45)
    ax[i_plot].legend([])
    ax[i_plot].set_ylabel('# births')
fig.tight_layout()

Plotting the yearly birth totals clearly shows a small increase after the First and a big boom after the Second World War.

In [ ]:
yearly_births = pd.pivot_table(births, index='birth_year', 
                               columns=['name_type', 'gender'], 
                               values='n_born', aggfunc='sum')

fig, ax = plt.subplots(2, 2, figsize=(10, 8), sharex='col', 
                       sharey='col')
for i_row, name_type in enumerate(['first', 'follow']):
    (yearly_births.sum()[name_type] / 1E3).plot(kind='bar', rot=45, 
                                                ax=ax[i_row, 0])
    ax[i_row, 0].set_ylabel('# births (thousands)')
    ax[i_row, 0].set_title('Total births for %s name' % name_type)
    
    (yearly_births[name_type] / 1E3).plot(ax=ax[i_row, 1])
    ax[i_row, 1].set_ylabel('# births (thousands)')
    ax[i_row, 1].set_title('Yearly births for %s name' % name_type)

Dutch baby boomers are more likely to have more than one follow name compared to previous and later generations, as the ratio of total follow and first names shows. 

In [ ]:
name_type_ratio = (yearly_births['follow'] / yearly_births['first'])
ax = name_type_ratio.plot(figsize=(8, 4))
ax.set_ylabel('Ratio')
ax.set_title("Ratio between # born with follow and first name")

## 2. Mortality rates 

The life expectancy is given as the number of people surviving from a cohort of 100,000.

In [ ]:
survival = pd.read_csv('data/cbs_life_expectancy.csv', delimiter=';', skiprows=4, 
                       names=['age', 'birth_year', 'gender', 'amount'], skipfooter=1)
survival.head()

Convert strings to floats and get rid of all the Dutch words.

In [ ]:
survival['age'] = survival.age.str.split(' ').apply(lambda x: float(x[0]))
survival['gender'] = survival.gender.map({'Mannen': 'male', 'Vrouwen': 'female'})
survival.ix[(survival.amount == '.'), 'amount'] = None
survival['survival_rate'] = survival['amount'].astype(np.float) / 100000.
survival.drop('amount', axis=1, inplace=True)
survival.head()

In [ ]:
survival.tail()

In [ ]:
MAX_AGE = 105


def ffill_missing_years(df):
    groupby_cols = ['gender', 'age']
    groups = df.groupby(groupby_cols)
    filled = groups.apply(ffill_group)
    return filled.drop(groupby_cols, axis=1).reset_index(groupby_cols)

def ffill_group(g):
    reindex_g = g.set_index('birth_year')
    filled_g = reindex_g.fillna(method='ffill')
    return filled_g.reset_index()

def interpolate_age_proba(df):
    new_axis = range(0, MAX_AGE + 1)
    groupby_cols = ['gender', 'birth_year']
    groups = df.groupby(groupby_cols)
    age_fcn = lambda x: interpolate_col(x, 'age', new_axis)
    interpolated = groups.apply(age_fcn)
    interpolated[interpolated.survival_rate < 0] = 0
    return interpolated.drop(groupby_cols, axis=1).reset_index(groupby_cols)

def interpolate_col(g, col, interpolate_axis):
    reindex_g = g.set_index(col).reindex(interpolate_axis)
    reindex_g.ix[max(interpolate_axis)] = 0
    interpolate_func = lambda x: pd.Series.interpolate(x, method='quadratic')
    interpolated_g = reindex_g.apply(interpolate_func)
    return interpolated_g.reset_index()

In [ ]:
filled_survival = ffill_missing_years(survival)
interpolated_survival = interpolate_age_proba(filled_survival)

In [ ]:
interpolated_survival.head()

In [ ]:
DEMO_YEAR = 1990
def get_yearly_data(df, year):
    """
    Get yearly data and pivot.
    
    :param df: DataFrame
    :param year: Target year
    :returns Pivoted DataFrame with data from year
    """
    temp = df[df.birth_year == year]
    data = temp.pivot(columns='gender', index='age', 
                      values='survival_rate')
    return data

year_original = get_yearly_data(survival, DEMO_YEAR)
year_filled = get_yearly_data(filled_survival, DEMO_YEAR)
year_interpolated = get_yearly_data(interpolated_survival, DEMO_YEAR)

fig, ax = plt.subplots(1, 3, figsize=(16, 4), sharex=True, sharey=True)

title_suffix = ' - year: ' + str(DEMO_YEAR)
year_original.plot(ax=ax[0], title='original' + title_suffix)
year_filled.plot(ax=ax[1], title='forward' + title_suffix)
year_interpolated.plot(ax=ax[2], title='interpolated' + title_suffix)
for current_ax in ax:
    current_ax.set_ylabel('survival rate')
    current_ax.legend(loc='lower left', frameon=True)

In [ ]:
TARGET_YEAR = 2014
assert (interpolated_survival.birth_year == TARGET_YEAR).any(), 'Invalid year'
is_alive_in_year = (interpolated_survival.age + 
                    interpolated_survival.birth_year == TARGET_YEAR)
year_survival = interpolated_survival[is_alive_in_year]

In [ ]:
births.head()

In [ ]:
year_survival.head()

In [ ]:
name_survival = pd.merge(births, year_survival)
name_survival['n_born_and_alive'] = name_survival.n_born * name_survival.survival_rate
name_survival['n_dead'] = name_survival.n_born - name_survival.n_born_and_alive

In [ ]:
name_survival[name_survival.name_type == 'first'].groupby('gender').n_born_and_alive.sum()

In [ ]:
#export.to_csv('out/name_survival.csv', index=False, encoding='utf-8')

In [ ]:
#name_survival.to_csv('out/name_survival.csv', index=False, encoding='utf-8')

In [ ]:
# See https://stackoverflow.com/questions/23433237/pandas-long-form-table-to-nested-json?rq=1

class NestedDict(dict):
    def __missing__(self, key):
        self[key] = NestedDict()
        return self[key]

In [ ]:
name_survival[['birth_year', 'n_born']] = name_survival[['birth_year', 'n_born']].applymap(np.round)
name_survival[['birth_year', 'n_born']] = name_survival[['birth_year', 'n_born']].astype(int)

In [ ]:
d = NestedDict()
ii = 0
for key, group in name_survival.groupby(['name', 'gender', 'name_type']):
    if ii == 0:
        d[key[0]][key[1]][key[2]]['data'] = group[['birth_year', 'n_born', 'n_born_and_alive']].to_dict(orient='records')
    ii += 1

In [ ]:
l = []
for key, group in name_survival.groupby(['name', 'gender', 'name_type']):
    data = group[['birth_year', 'n_born', 'n_born_and_alive']].to_dict(orient='records')
    entry = dict(zip(('name', 'gender', 'name_type', 'data'), list(key) + [data]))
    l.append(entry)

In [ ]:
df = pd.DataFrame(l)
df.to_csv('test.csv', index=False, encoding='utf-8')

In [ ]:
!ls -lh

<code>

sudo service mongodb stop
mongod --dbpath ~/temp/mongo_data/

use names
db.data.remove()

mongoimport -d names -c data --type json --file dutch-names/test.json
mongoimport -d names -c data --type csv test.csv -headerline

In [ ]:
group[['birth_year', 'n_born', 'n_born_and_alive']].to_dict(orient='records')

In [ ]:
import json


In [ ]:
with open('out/anne.json', 'w') as f:
    json.dump(d['Anne'], f)

In [ ]:
with open('out/names_export.json', 'w') as f:
    json.dump(d, f)


<code>
[
    {
    'name': 'Aaf',
    'gender': 'female'
    'data':
        {
        'first': 
            [
                {
                'year': 2015,
                'n_born': 24,
                'n_born_and_alive: 20,
                },
                {
                'year': 2010,
                'n_born': 10,
                'n_born_and_alive': 2,
                },
            ]
        'follow': 
            [
                {
                }
                {
                }
            ]
        }

            }
    },
    {
    'name': 'Anne' 
    },
]
</code>

In [ ]:
group[['birth_year', 'n_born']].to_dict(orient='records')
#pd.DataFrame.to_dict()

<code>mongoimport -d names -c data --type csv --file names/out/name_survival.csv -headerline</code>

<code>mongod --dbpath ~/temp/mongo_data/</code>


In [ ]:
henk = name_survival[(name_survival.name == u'Daniël') & (name_survival.gender == 'male') &
                     (name_survival.name_type == 'first')]
henk.set_index('birth_year')[['n_born', 'n_born_and_alive']].plot()
henk = name_survival[(name_survival.name == 'Hendrikus') & (name_survival.gender == 'male') &
                     (name_survival.name_type == 'first')]
henk.set_index('birth_year')[['n_born', 'n_born_and_alive']].plot()

In [ ]:
henk = name_survival[(name_survival.name == u'Daniël') & (name_survival.gender == 'male') &
                     (name_survival.name_type == 'first')]
henk.set_index('birth_year')[['n_born', 'n_born_and_alive', 'n_dead']].plot()
henk = name_survival[(name_survival.name == u'Willem') & (name_survival.gender == 'male') &
                     (name_survival.name_type == 'first')]
henk.set_index('birth_year')[['n_born', 'n_born_and_alive', 'n_dead']].plot()

In [ ]:
pd.Series(export.name.unique()).to_json('out/names.json', orient='values')

### Ideas:
<li> Total deaths per name
<li> Cumulative deaths vs cumulative born